In [121]:
import pandas as pd
from sklearn import preprocessing, linear_model, pipeline, model_selection, feature_selection, tree
from shared import get_production_lines
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [122]:
paint_per_date_df = pd.read_feather('../data/paint_per_date.feather').reset_index()
paint_per_date_df = paint_per_date_df.drop(
    ['shreveport_yield', 'canning_yield', 'yield_difference', 'yield_ratio', 'abs_yield_difference',
     'paint_catalog_yield',
     'monetary_value_difference_usd', 'paint_name', 'date', 'month', 'user'], axis=1)

In [123]:
transformed_ppd_df = paint_per_date_df.assign(
    input_weight_kg=np.sqrt(paint_per_date_df['input_weight_kg']),
    weight_kg=np.sqrt(paint_per_date_df['weight_kg']),
    length_m=np.sqrt(paint_per_date_df['length_m']),
    m2=np.sqrt(paint_per_date_df['m2']),
    polyethylene_applied_meters=np.sqrt(paint_per_date_df['polyethylene_applied_meters']),
    width_mm=np.square(paint_per_date_df['width_mm']),
    object_monetary_value=np.sqrt(paint_per_date_df['object_monetary_value']),
    monetary_value_usd=np.sqrt(paint_per_date_df['monetary_value_usd']),
    total_liters_used=np.sqrt(paint_per_date_df['total_liters_used']),
    price_per_liter=np.log(paint_per_date_df['price_per_liter']),
    finish_coat_sup=np.square(paint_per_date_df['finish_coat_sup']),
    finish_coat_lo_sup=np.square(paint_per_date_df['finish_coat_lo_sup']),
    finish_coat_lm_sup=np.square(paint_per_date_df['finish_coat_lm_sup']),
    solvent_resistance_inf=np.sqrt(paint_per_date_df['solvent_resistance_inf']),
    terminc_emissivity=np.square(paint_per_date_df['terminc_emissivity']),
    density=np.sqrt(paint_per_date_df['density']),
    weight_input_weight=lambda df: df['input_weight_kg'] * df['weight_kg'],
    length_m_m2=lambda df: df['length_m'] * df['m2'],
    object_monetary_value_input_weight_kg=lambda df: df['object_monetary_value'] * df['input_weight_kg'],
    object_monetary_value_weight_kg=lambda df: df['object_monetary_value'] * df['weight_kg'],
    object_monetary_value_length_m=lambda df: df['object_monetary_value'] * df['length_m'],
    object_monetary_value_m2=lambda df: df['object_monetary_value'] * df['m2'],
    monetary_value_usd_input_weight_kg=lambda df: df['monetary_value_usd'] * df['input_weight_kg'],
    monetary_value_usd_weight_kg=lambda df: df['monetary_value_usd'] * df['weight_kg'],
    monetary_value_usd_length_m=lambda df: df['monetary_value_usd'] * df['length_m'],
    monetary_value_usd_m2=lambda df: df['monetary_value_usd'] * df['m2'],
    total_liters_used_input_weight_kg=lambda df: df['total_liters_used'] * df['input_weight_kg'],
    total_liters_used_weight_kg=lambda df: df['total_liters_used'] * df['weight_kg'],
    total_liters_used_length_m=lambda df: df['total_liters_used'] * df['length_m'],
    total_liters_used_m2=lambda df: df['total_liters_used'] * df['m2'],
    transaction_monetary_value_input_weight_kg=lambda df: df['transaction_monetary_value'] * df['input_weight_kg'],
    transaction_monetary_value_weight_kg=lambda df: df['transaction_monetary_value'] * df['weight_kg'],
    transaction_monetary_value_length_m=lambda df: df['transaction_monetary_value'] * df['length_m'],
    transaction_monetary_value_m2=lambda df: df['transaction_monetary_value'] * df['m2'],
    object_monetary_value_monetary_value_usd=lambda df: df['object_monetary_value'] * df['monetary_value_usd'],
    object_monetary_value_total_liters_used=lambda df: df['object_monetary_value'] * df['total_liters_used'],
    object_monetary_value_transaction_monetary_value=lambda df: df['object_monetary_value'] * df[
        'transaction_monetary_value'],
    monetary_value_usd_total_liters_used=lambda df: df['monetary_value_usd'] * df['total_liters_used'],
    monetary_value_usd_transaction_monetary_value=lambda df: df['monetary_value_usd'] * df[
        'transaction_monetary_value'],
    total_liters_used_transaction_monetary_value=lambda df: df['total_liters_used'] * df['transaction_monetary_value'],
    metal_temp_finish_coat_sup=lambda df: df['metal_temp'] * df['finish_coat_sup'],
    metal_temp_finish_coat_lo_sup=lambda df: df['metal_temp'] * df['finish_coat_lo_sup'],
    metal_temp_finish_coat_lm_sup=lambda df: df['metal_temp'] * df['finish_coat_lm_sup'],
    solids_per_min_volume_width_mm=lambda df: df['solids_per_min_volume'] * df['width_mm'],
    solids_per_min_volume_avg_thickness_mm=lambda df: df['solids_per_min_volume'] * df['avg_thickness_mm'],
    solid_by_weight_width_mm=lambda df: df['solid_by_weight'] * df['width_mm'],
    solid_by_weight_avg_thickness_mm=lambda df: df['solid_by_weight'] * df['avg_thickness_mm'],
    solid_by_weight_solvent_resistance_inf=lambda df: df['solid_by_weight'] * df['solvent_resistance_inf'],
    solid_by_volume_avg_thickness_mm=lambda df: df['solid_by_volume'] * df['avg_thickness_mm'],
    solid_by_volume_width_mm=lambda df: df['solid_by_volume'] * df['width_mm'],
    solid_by_volume_solvent_resistance_inf=lambda df: df['solid_by_volume'] * df['solvent_resistance_inf'],
    density_width_mm=lambda df: df['density'] * df['width_mm'],
    avg_thickness_mm_recommended_rev=lambda df: df['avg_thickness_mm'] * df['recommended_rev'],
    solvent_resistance_inf_solids_per_min_volume=lambda df: df['solvent_resistance_inf'] * df['solids_per_min_volume'],
    solvent_resistance_inf_nominal_dry_coat=lambda df: df['solvent_resistance_inf'] * df['nominal_dry_coat'],
    solvent_resistance_inf_paint_dry_coat_min=lambda df: df['solvent_resistance_inf'] * df['paint_dry_coat_min'],
    solvent_resistance_inf_paint_dry_coat_max=lambda df: df['solvent_resistance_inf'] * df['paint_dry_coat_max'],
    max_weight_density=lambda df: df['max_weight'] * df['density'],
    max_weight_solid_by_weight=lambda df: df['max_weight'] * df['solid_by_weight'],
    max_weight_metal_temp=lambda df: df['max_weight'] * df['metal_temp'],
    max_weight_tone_analysis_de_inf=lambda df: df['max_weight'] * df['tone_analysis_de_inf'],
    max_weight_tone_analysis_de_sup=lambda df: df['max_weight'] * df['tone_analysis_de_sup'],
    max_weight_min_weight=lambda df: df['max_weight'] * df['min_weight'],
    tone_analysis_de_inf_finish_coat_sup=lambda df: df['tone_analysis_de_inf'] * df['finish_coat_sup'],
    tone_analysis_de_inf_finish_coat_lo_sup=lambda df: df['tone_analysis_de_inf'] * df['finish_coat_lo_sup'],
    tone_analysis_de_inf_finish_coat_lm_sup=lambda df: df['tone_analysis_de_inf'] * df['finish_coat_lm_sup'],
    tone_analysis_de_sup_finish_coat_sup=lambda df: df['tone_analysis_de_sup'] * df['finish_coat_sup'],
    tone_analysis_de_sup_finish_coat_lo_sup=lambda df: df['tone_analysis_de_sup'] * df['finish_coat_lo_sup'],
    tone_analysis_de_sup_finish_coat_lm_sup=lambda df: df['tone_analysis_de_sup'] * df['finish_coat_lm_sup'],
    expected_yield_nominal_dry_coat=lambda df: df['expected_yield'] * df['nominal_dry_coat'],
    expected_yield_solids_per_min_volume=lambda df: df['expected_yield'] * df['solids_per_min_volume'],
    expected_yield_paint_dry_coat_max=lambda df: df['expected_yield'] * df['paint_dry_coat_max'],
    expected_yield_paint_dry_coat_min=lambda df: df['expected_yield'] * df['paint_dry_coat_min'],
    expected_yield_viscosity=lambda df: df['expected_yield'] * df['viscosity'],
    expected_yield_metal_temp=lambda df: df['expected_yield'] * df['metal_temp'],
    expected_yield_solid_by_volume=lambda df: df['expected_yield'] * df['solid_by_volume'],
    expected_yield_solid_by_weight=lambda df: df['expected_yield'] * df['solid_by_weight'],
    expected_yield_density=lambda df: df['expected_yield'] * df['density'],
    expected_yield_price_per_liter=lambda df: df['expected_yield'] * df['price_per_liter'],
    expected_yield_transaction_monetary_value=lambda df: df['expected_yield'] * df['transaction_monetary_value'],
    expected_yield_monetary_value_usd=lambda df: df['expected_yield'] * df['monetary_value_usd'],
    expected_yield_object_monetary_value=lambda df: df['expected_yield'] * df['object_monetary_value'],
    expected_yield_tone_analysis_de_sup=lambda df: df['expected_yield'] * df['tone_analysis_de_sup'],
    expected_yield_tone_analysis_de_inf=lambda df: df['expected_yield'] * df['tone_analysis_de_inf'],
    expected_yield_solvent_resistance_sup=lambda df: df['expected_yield'] * df['solvent_resistance_sup'],
    expected_yield_finish_coat_sup=lambda df: df['expected_yield'] * df['finish_coat_sup'],
    expected_yield_finish_coat_lo_sup=lambda df: df['expected_yield'] * df['finish_coat_lo_sup'],
    expected_yield_finish_coat_lm_sup=lambda df: df['expected_yield'] * df['finish_coat_lm_sup'],
    expected_yield_min_weight=lambda df: df['expected_yield'] * df['min_weight'],
    expected_yield_max_weight=lambda df: df['expected_yield'] * df['max_weight'],
)

/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [124]:
p1, p2 = get_production_lines(transformed_ppd_df)
p1 = p1.drop(['production_line'], axis=1).fillna(0)
p2 = p2.drop(['production_line'], axis=1).fillna(0)


/home/hiram/Projects/Pandalytics/Evidencia 3/shared.py:46: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  groups = df.groupby(['production_line'])
/home/hiram/Projects/Pandalytics/Evidencia 3/shared.py:47: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  p1_df = groups.get_group('Pintado 1')
/home/hiram/Projects/Pandalytics/Evidencia 3/shared.py:48: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  p2_df = groups.get_group('Pintado 2')


In [125]:
X1 = p1.drop(['real_yield'], axis=1)
Y1 = p1['real_yield']

X2 = p2.drop(['real_yield'], axis=1)
Y2 = p2['real_yield']

elasticnet_pipe = pipeline.Pipeline(
    [
        ('scaler', preprocessing.MinMaxScaler()),
        ('variance_thresh', feature_selection.VarianceThreshold()),
        ('model', linear_model.ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], selection='random'))
    ]
)

elasticnet_results_1 = pd.DataFrame(
    model_selection.cross_validate(elasticnet_pipe, X1, Y1, scoring=('r2', 'neg_mean_squared_error'),
                                   return_estimator=True))
# lassolars_results_1 = pd.DataFrame(model_selection.cross_validate(lassolars_pipe, X1, Y1, scoring=('r2', 'neg_mean_squared_error'), return_estimator=True))
# ridge_results_1 = pd.DataFrame(model_selection.cross_validate(ridge_pipe, X1, Y1, scoring=('r2', 'neg_mean_squared_error'), return_estimator=True))

elasticnet_results_2 = pd.DataFrame(
    model_selection.cross_validate(elasticnet_pipe, X2, Y2, scoring=('r2', 'neg_mean_squared_error'),
                                   return_estimator=True))
# lassolars_results_2 = pd.DataFrame(model_selection.cross_validate(lassolars_pipe, X2, Y2, scoring=('r2', 'neg_mean_squared_error'), return_estimator=True))
# ridge_results_2 = pd.DataFrame(model_selection.cross_validate(ridge_pipe, X2, Y2, scoring=('r2', 'neg_mean_squared_error'), return_estimator=True))


/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 914.4961074413732, tolerance: 487.36464100528735
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 693.9797610677779, tolerance: 487.36464100528735
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1116.3124771472067, tolerance: 487.36464100528735
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Pro

In [126]:
elasticnet_results_1

fit_time  score_time                                          estimator  \
0  7.094120    0.004886  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
1  3.990371    0.004045  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
2  3.700158    0.002937  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
3  4.001385    0.004846  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
4  4.374537    0.003349  (MinMaxScaler(), VarianceThreshold(), ElasticN...   

    test_r2  test_neg_mean_squared_error  
0  0.554004                  -540.435509  
1  0.463657                 -1246.564004  
2 -0.028213                 -3535.618871  
3  0.507636                  -447.186494  
4  0.617659                 -1012.244671

In [127]:
elasticnet_results_2

fit_time  score_time                                          estimator  \
0  2.743662    0.002893  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
1  3.924375    0.002816  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
2  3.211801    0.002919  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
3  3.146456    0.004764  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
4  3.607070    0.004578  (MinMaxScaler(), VarianceThreshold(), ElasticN...   

    test_r2  test_neg_mean_squared_error  
0  0.697017                  -780.192254  
1  0.593991                 -3332.599293  
2  0.655640                 -1127.456646  
3  0.480373                 -1089.426406  
4 -0.066857                 -1323.913659

In [128]:
elasticnet_results = pd.DataFrame(
    model_selection.cross_validate(elasticnet_pipe, transformed_ppd_df.drop(columns=['real_yield', 'production_line']).fillna(0), transformed_ppd_df['real_yield'], scoring=('r2', 'neg_mean_squared_error'),
                                   return_estimator=True))
# lassolars_results_1

/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1629.9053907436319, tolerance: 1027.9837513632856
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1187.5086615015753, tolerance: 854.5537168415888
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Projects/Pandalytics/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:664: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2297.4123173425905, tolerance: 854.5537168415888
  model = cd_fast.enet_coordinate_descent_gram(
/home/hiram/Pro

In [129]:
elasticnet_results

fit_time  score_time                                          estimator  \
0  5.772328    0.008449  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
1  4.685064    0.007557  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
2  4.648921    0.007198  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
3  4.119514    0.004557  (MinMaxScaler(), VarianceThreshold(), ElasticN...   
4  4.481069    0.007261  (MinMaxScaler(), VarianceThreshold(), ElasticN...   

    test_r2  test_neg_mean_squared_error  
0  0.606086                  -828.026062  
1  0.627655                 -1849.657821  
2  0.326530                 -2401.134334  
3  0.681743                  -381.856335  
4  0.609599                  -806.993414

In [130]:
# lassolars_results_2

In [131]:
# ridge_results_1

In [132]:
# ridge_results_2